# Bank Customer Churn Prediction (part 4 - GridSearchCV and RandomizedSearchCV)

Link to part 3 - https://github.com/Maxstef/data-loves-ml-for-people-course/blob/main/notebooks/2_3_decision_tree/0_1_bank_customer_decision_tree.ipynb



## Imports & Files downloads

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np

from mlpeople.io.google_drive import download_file_iss

# download files from google drive
download_file_iss('1Xz-cqp6y-Y_oCiaSJeNsbU3-o5lM-5wY', output_path='./downloads/train.csv')
download_file_iss('1jsg2iPVphDMiHCEyqQeDSc4yDcMB3-r_', output_path='./downloads/test.csv')
download_file_iss('1tv0beG2n8cUQ6KcdaXhFzQI_Fd9640gg', output_path='./downloads/sample_submission.csv')

# read train.csv / show first 5 rows / show info
raw_df = pd.read_csv('downloads/train.csv', index_col=0)
target_col = "Exited"
raw_df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
id,,,,,,,,,,,,,
0,15779985.0,Nwankwo,678.0,France,Male,29.0,4.0,0.00,3.0,1.0,0.0,180626.36,0.0
1,15650086.0,Ch'in,687.0,France,Female,34.0,1.0,0.00,2.0,0.0,1.0,63736.17,0.0
2,15733602.0,Thompson,682.0,France,Female,52.0,6.0,0.00,3.0,0.0,0.0,179655.87,1.0
3,15645794.0,Macleod,753.0,Germany,Male,44.0,6.0,83347.25,2.0,1.0,0.0,161407.48,0.0
4,15633840.0,Hsia,544.0,Germany,Female,55.0,0.0,107747.57,1.0,1.0,0.0,176580.86,1.0


## Best previous

In [ ]:
# methods used previously for experiments
from mlpeople.optimization.experiments.polynomial_pipeline_experiments import run_experiment_poly, run_experiments_poly

### Log Reg

In [3]:
from math import inf

result_2 = run_experiment_poly(
    raw_df,
    target_col,
    stratify_col=target_col,
    drop_cols=["Surname", "CustomerId", "Tenure", "HasCrCard"],
    scale_mode="standard",
    encode_drop=None,
    separate_binary_numeric=False,
    polynomial_degree=4,
    polynomial_after_scale=True,
    top_n_cat_values=None,
    binary_cat_flag_cols={
        'Geography': [{'flag_name': 'IsGermany',
            'value': 'Germany',
            'drop_original': True
        }]
    },
    binary_num_flag_cols=None,
    polynomial_interaction_only=False,
    model=LogisticRegression(C=0.5, solver='liblinear'),
    num_bin_cols={
        "EstimatedSalary": {
            "bins": [0, 25000, 100000, inf],
            "labels": ["low", "medium", "high"],
            "new_col": "SalaryScore",
            "drop_original": True
        },
        "CreditScore": {
            "bins": [0, 675, inf],
            "labels": ["low", "high"],
            "new_col": "CreditScoreScore",
            "drop_original": True
        }
    }
)

best_lg_roc_auc_train = result_2["roc_auc_train"]
best_lg_roc_auc_val = result_2["roc_auc_val"]
best_lg_poly_model_pipeline = result_2["pipeline"]

best_lg_roc_auc_train, best_lg_roc_auc_val

(0.9381483220937081, 0.9395325468139104)

In [4]:
best_lg_poly_model_pipeline

,steps,"[('num_bin', ...), ('cat_flags', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,mapping,"{'CreditScore': {'bins': [0, 675, ...], 'drop_original': True, 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore'}, 'EstimatedSalary': {'bins': [0, 25000, ...], 'drop_original': True, 'labels': ['low', 'medium', ...], 'new_col': 'SalaryScore'}}"
,flag_config,"{'Geography': [{'drop_original': True, 'flag_name': 'IsGermany', 'value': 'Germany'}]}"
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None


### Decision Tree

In [5]:
result_3 = run_experiment_poly(
    raw_df,
    target_col,
    stratify_col=target_col,
    drop_cols=["Surname", "CustomerId"],
    scale_mode="standard",
    encode_drop=None,
    separate_binary_numeric=False,
    polynomial_degree=1,
    polynomial_after_scale=True,
    top_n_cat_values=None,
    binary_cat_flag_cols=None,
    binary_num_flag_cols=None,
    polynomial_interaction_only=False,
    model=DecisionTreeClassifier(random_state=42, max_depth=5),
    num_bin_cols=None
)

best_dt_roc_auc_train = result_3["roc_auc_train"]
best_dt_roc_auc_val = result_3["roc_auc_val"]
best_dt_poly_model_pipeline = result_3["pipeline"]

best_dt_roc_auc_train, best_dt_roc_auc_val

(0.9256944626530315, 0.9219267439467729)

In [6]:
best_dt_poly_model_pipeline

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The

## Pipelines for experiments

Create pipelines that include previosly implemented preprocessing steps. This time we use RandomizedSearchCV and GridSearchCV with cross-validation included for looking best models hyperparams and preprocessing options. The final models are trained on all available train data.

In [7]:
# raw data for experiments
X_raw = raw_df.drop([target_col, "CustomerId", "Surname"],axis=1)
y_raw = raw_df[target_col]

In [ ]:
# Pipeline that includes previosly implemented preprocessing steps
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder

from mlpeople.preprocessing.transformers import (
    ColumnDropper,
    NumericBinaryFlagTransformer,
    CategoricalBinaryFlagTransformer,
    TopNCategoricalTransformer,
    NumericBinner,
)

numeric_selector = make_column_selector(dtype_include=np.number)
categorical_selector = make_column_selector(dtype_exclude=np.number)

numeric_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("poly_features", PolynomialFeatures(include_bias=False))
])

categorical_pipeline = Pipeline(
    [("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="if_binary"))]
)

preprocessor = ColumnTransformer(
    [
        ("num", numeric_pipeline, numeric_selector),
        ("cat", categorical_pipeline, categorical_selector),
    ]
)

# no need to scale numeric and create polinomial features for decistion tree
preprocessor_dt = ColumnTransformer(
    [
        ("num", "passthrough", numeric_selector),
        ("cat", categorical_pipeline, categorical_selector),
    ]
)

# pipeline for log reg
pipeline_lg = Pipeline([
    ("drop_cols", ColumnDropper([])),
    ("num_flags", NumericBinaryFlagTransformer({})),
    ("num_bin", NumericBinner({})),
    ("cat_flags", CategoricalBinaryFlagTransformer({})),
    ("top_n", TopNCategoricalTransformer({})),
    ("preprocessing", preprocessor),
    ("model", LogisticRegression(solver='liblinear'))
])

# pipeline for decision tree
pipeline_dt = Pipeline([
    ("drop_cols", ColumnDropper([])),
    ("preprocessing", preprocessor_dt),
    ("model", DecisionTreeClassifier(random_state=42, max_depth=5))
])

## GridSearchCV Experiments

### Log Reg - Drop cols

In [18]:
%%time

# previously it was determined that 20-40% is optimal for validataion data so n_splits=3 should be fine
skf = StratifiedKFold(n_splits=3, shuffle=False)

grid_search_lg_dc = GridSearchCV(
    pipeline_lg,
    {
        "drop_cols__columns": [
            [],
            ["Tenure"],
            ["Tenure", "HasCrCard"],
            ["Tenure", "HasCrCard", "EstimatedSalary"],
            ["Tenure", "HasCrCard", "CreditScore"],
            ["Tenure", "HasCrCard", "EstimatedSalary", "CreditScore"],
        ],
        "preprocessing__num__poly_features__degree": [1, 2, 3],
    },
    cv=skf,
    scoring="roc_auc",
)

grid_search_lg_dc.fit(X_raw, y_raw)

CPU times: user 5.09 s, sys: 168 ms, total: 5.26 s
Wall time: 5.28 s


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...liblinear'))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'drop_cols__columns': [[], ['Tenure'], ...], 'preprocessing__num__poly_features__degree': [1, 2, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",StratifiedKFo...shuffle=False)
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fo

In [19]:
import pandas as pd

grid_results_lg_dc = pd.DataFrame(grid_search_lg_dc.cv_results_)
grid_results_view_lg_dc = grid_results_lg_dc[
    [
        "param_drop_cols__columns",
        "param_preprocessing__num__poly_features__degree",
        "mean_test_score",
        "std_test_score",
        "rank_test_score",
    ]
].sort_values("mean_test_score", ascending=False)

grid_results_view_lg_dc

,param_drop_cols__columns,param_preprocessing__num__poly_features__degree,mean_test_score,std_test_score,rank_test_score
11,"[Tenure, HasCrCard, EstimatedSalary]",3,0.934083,0.001249,1
17,"[Tenure, HasCrCard, EstimatedSalary, CreditScore]",3,0.933816,0.001197,2
8,"[Tenure, HasCrCard]",3,0.933100,0.001208,3
14,"[Tenure, HasCrCard, CreditScore]",3,0.933057,0.001104,4
5,[Tenure],3,0.932196,0.000963,5
2,[],3,0.930139,0.001635,6
10,"[Tenure, HasCrCard, EstimatedSalary]",2,0.929596,0.000976,7
7,"[Tenure, HasCrCard]",2,0.929377,0.000963,8
4,[Tenure],2,0.929245,0.001172,9
16,"[Tenure, HasCrCard, EstimatedSalary, CreditScore]",2,0.929042,0.000912,10


 - Dropping low correlated cols provides better result and less noise
 - Tenure, HasCrCard seems to bring only noise
 - EstimatedSalary, CreditScore cols better to explore more
 - Polinomial degree at least 3 should be used

### Log Reg - Polynomial degree

In [20]:
grid_search_lg_pd = GridSearchCV(
    pipeline_lg,
    {
        "drop_cols__columns": [
            ["Tenure", "EstimatedSalary", "HasCrCard", "CreditScore"]
        ],
        "preprocessing__num__poly_features__degree": range(1,7),
    },
    cv=skf,
    scoring="roc_auc",
)

grid_search_lg_pd.fit(X_raw, y_raw)

/opt/miniconda3/envs/mlpeople4/lib/python3.12/site-packages/sklearn/svm/_base.py:1258: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda3/envs/mlpeople4/lib/python3.12/site-packages/sklearn/svm/_base.py:1258: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda3/envs/mlpeople4/lib/python3.12/site-packages/sklearn/svm/_base.py:1258: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...liblinear'))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'drop_cols__columns': [['Tenure', 'EstimatedSalary', ...]], 'preprocessing__num__poly_features__degree': range(1, 7)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",StratifiedKFo...shuffle=False)
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation t

In [21]:
import pandas as pd

grid_results_lg_pd = pd.DataFrame(grid_search_lg_pd.cv_results_)
grid_results_view_lg_pd = grid_results_lg_pd[
    [
        "param_drop_cols__columns",
        "param_preprocessing__num__poly_features__degree",
        "mean_test_score",
        "std_test_score",
        "rank_test_score",
    ]
].sort_values("mean_test_score", ascending=False)

grid_results_view_lg_pd

,param_drop_cols__columns,param_preprocessing__num__poly_features__degree,mean_test_score,std_test_score,rank_test_score
2,"[Tenure, EstimatedSalary, HasCrCard, CreditScore]",3,0.933816,0.001197,1
3,"[Tenure, EstimatedSalary, HasCrCard, CreditScore]",4,0.932915,0.000922,2
4,"[Tenure, EstimatedSalary, HasCrCard, CreditScore]",5,0.932247,0.000702,3
5,"[Tenure, EstimatedSalary, HasCrCard, CreditScore]",6,0.930372,0.001352,4
1,"[Tenure, EstimatedSalary, HasCrCard, CreditScore]",2,0.929042,0.000912,5
0,"[Tenure, EstimatedSalary, HasCrCard, CreditScore]",1,0.879621,0.005445,6


 - 3 is better than 4 this time

### Log Reg - Polynomial degree and drop cols combined

In [22]:
%%time
grid_search_lg_pd_dc = GridSearchCV(
    pipeline_lg,
    {
        "drop_cols__columns": [
            ["Tenure", "EstimatedSalary", "HasCrCard", "CreditScore"],
            ["Tenure", "EstimatedSalary", "HasCrCard",],
            ["Tenure", "EstimatedSalary", "CreditScore"],
            ["Tenure", "EstimatedSalary",],
        ],
        "preprocessing__num__poly_features__degree": [3,4],
    },
    cv=skf,
    scoring="roc_auc",
)

grid_search_lg_pd_dc.fit(X_raw, y_raw)

CPU times: user 16.7 s, sys: 413 ms, total: 17.1 s
Wall time: 18.1 s


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...liblinear'))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'drop_cols__columns': [['Tenure', 'EstimatedSalary', ...], ['Tenure', 'EstimatedSalary', ...], ...], 'preprocessing__num__poly_features__degree': [3, 4]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",StratifiedKFo...shuffle=False)
,"verbose verbose: intControls the verbosity: the higher, the mo

In [23]:
import pandas as pd

grid_results_lg_pd_dc = pd.DataFrame(grid_search_lg_pd_dc.cv_results_)
grid_results_view_lg_pd_dc = grid_results_lg_pd_dc[
    [
        "param_drop_cols__columns",
        "param_preprocessing__num__poly_features__degree",
        "mean_test_score",
        "std_test_score",
        "rank_test_score",
    ]
].sort_values("mean_test_score", ascending=False)

grid_results_view_lg_pd_dc

,param_drop_cols__columns,param_preprocessing__num__poly_features__degree,mean_test_score,std_test_score,rank_test_score
2,"[Tenure, EstimatedSalary, HasCrCard]",3,0.934083,0.001249,1
0,"[Tenure, EstimatedSalary, HasCrCard, CreditScore]",3,0.933816,0.001197,2
6,"[Tenure, EstimatedSalary]",3,0.933542,0.001298,3
4,"[Tenure, EstimatedSalary, CreditScore]",3,0.933406,0.001081,4
1,"[Tenure, EstimatedSalary, HasCrCard, CreditScore]",4,0.932915,0.000922,5
5,"[Tenure, EstimatedSalary, CreditScore]",4,0.931445,0.000716,6
3,"[Tenure, EstimatedSalary, HasCrCard]",4,0.931307,0.000049,7
7,"[Tenure, EstimatedSalary]",4,0.929313,0.001001,8


 - 3 better than 4 for all options

### Log Reg - NumericBinner and CategoricalBinaryFlag Custom Transformers

In [24]:
cat_flags_config = {
    "Geography": [{"flag_name": "IsGermany",
        "value": "Germany",
        "drop_original": True
    }]
}

num_bin_mapping = {
    "EstimatedSalary": {
        "bins": [0, 25000, 100000, inf],
        "labels": ["low", "medium", "high"],
        "new_col": "SalaryScore",
        "drop_original": True
    },
    "CreditScore": {
        "bins": [0, 675, inf],
        "labels": ["low", "high"],
        "new_col": "CreditScoreScore",
        "drop_original": True
    }
}

In [25]:
%%time

grid_search_lg_nb_cf_ct = GridSearchCV(
    pipeline_lg,
    {
        "cat_flags__flag_config": [
            {}, 
            cat_flags_config
        ],
        "num_bin__mapping": [
            {},
            num_bin_mapping
        ],
        "drop_cols__columns": [
            ["Tenure", "HasCrCard"],
        ],
        "preprocessing__num__poly_features__degree": [3,4],
    },
    cv=skf,
    scoring="roc_auc",
)

grid_search_lg_nb_cf_ct.fit(X_raw, y_raw)

CPU times: user 28.1 s, sys: 323 ms, total: 28.5 s
Wall time: 28.7 s


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...liblinear'))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'cat_flags__flag_config': [{}, {'Geography': [{'drop_original': True, 'flag_name': 'IsGermany', 'value': 'Germany'}]}], 'drop_cols__columns': [['Tenure', 'HasCrCard']], 'num_bin__mapping': [{}, {'CreditScore': {'bins': [0, 675, ...], 'drop_original': True, 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore'}, 'EstimatedSalary': {'bins': [0, 25000, ...], 'drop_original': True, 'labels': ['low', 'medium', ...], 'new_col': 'SalaryScore'}}], 'preprocessing__num__poly_features__degree': [3, 4]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are in

In [26]:
import pandas as pd

grid_results_lg_nb_cf_ct = pd.DataFrame(grid_search_lg_nb_cf_ct.cv_results_)
grid_results_view_lg_nb_cf_ct = grid_results_lg_nb_cf_ct[
    [
        "param_cat_flags__flag_config",
        "param_num_bin__mapping",
        "param_preprocessing__num__poly_features__degree",
        "mean_test_score",
        "std_test_score",
        "rank_test_score",
    ]
].sort_values("mean_test_score", ascending=False)

grid_results_view_lg_nb_cf_ct

,param_cat_flags__flag_config,param_num_bin__mapping,param_preprocessing__num__poly_features__degree,mean_test_score,std_test_score,rank_test_score
6,"{'Geography': [{'flag_name': 'IsGermany', 'val...","{'EstimatedSalary': {'bins': [0, 25000, 100000...",3,0.936280,0.000912,1
4,"{'Geography': [{'flag_name': 'IsGermany', 'val...",{},3,0.934563,0.001154,2
2,{},"{'EstimatedSalary': {'bins': [0, 25000, 100000...",3,0.934527,0.001262,3
7,"{'Geography': [{'flag_name': 'IsGermany', 'val...","{'EstimatedSalary': {'bins': [0, 25000, 100000...",4,0.934196,0.000420,4
3,{},"{'EstimatedSalary': {'bins': [0, 25000, 100000...",4,0.933664,0.000968,5
0,{},{},3,0.933100,0.001208,6
1,{},{},4,0.927865,0.000915,7
5,"{'Geography': [{'flag_name': 'IsGermany', 'val...",{},4,0.926128,0.001993,8


In [58]:
# update pipeline for log reg based on above experiments
pipeline_lg = Pipeline([
    ("drop_cols", ColumnDropper(["Tenure", "HasCrCard"])),
    ("num_flags", NumericBinaryFlagTransformer({})),
    ("num_bin", NumericBinner(num_bin_mapping)),
    ("cat_flags", CategoricalBinaryFlagTransformer(cat_flags_config)),
    ("top_n", TopNCategoricalTransformer({})),
    ("preprocessing", preprocessor),
    ("model", LogisticRegression(solver='liblinear'))
])

 - EstimatedSalary and CreditScore still have some usefull details in case we bin them customly
 - only germany details is enough from geography col

### Log Reg - Model Hyperparams

In [29]:
%%time

grid_search_lg_mhp = GridSearchCV(
    pipeline_lg,
    {
        "preprocessing__num__poly_features__degree": [3],
        "model": [
            LogisticRegression(C=0.25, solver='liblinear'),
            LogisticRegression(C=0.5, solver='liblinear'),
            LogisticRegression(C=1, solver='liblinear'),
            LogisticRegression(C=5, solver='liblinear'),
            LogisticRegression(C=0.5, solver='liblinear', l1_ratio=1.0),
            LogisticRegression(C=1, solver='liblinear', l1_ratio=1.0),
            LogisticRegression(C=5, solver='liblinear', l1_ratio=1.0),
        ],
    },
    cv=skf,
    scoring="roc_auc",
)

grid_search_lg_mhp.fit(X_raw, y_raw)

CPU times: user 31.9 s, sys: 398 ms, total: 32.3 s
Wall time: 33.2 s


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...liblinear'))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'model': [LogisticRegre...r='liblinear'), LogisticRegre...r='liblinear'), ...], 'preprocessing__num__poly_features__degree': [3]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",StratifiedKFo...shuffle=False)
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the c

In [30]:
grid_results_lg_mhp = pd.DataFrame(grid_search_lg_mhp.cv_results_)
grid_results_view_lg_mhp = grid_results_lg_mhp[
    [
        "param_model",
        "param_preprocessing__num__poly_features__degree",
        "mean_test_score",
        "std_test_score",
        "rank_test_score",
    ]
].sort_values("mean_test_score", ascending=False)

grid_results_view_lg_mhp

,param_model,param_preprocessing__num__poly_features__degree,mean_test_score,std_test_score,rank_test_score
4,"LogisticRegression(C=0.5, l1_ratio=1.0, solver...",3,0.936450,0.000770,1
5,"LogisticRegression(C=1, l1_ratio=1.0, solver='...",3,0.936330,0.000901,2
1,"LogisticRegression(C=0.5, solver='liblinear')",3,0.936311,0.000790,3
2,"LogisticRegression(C=1, solver='liblinear')",3,0.936280,0.000912,4
0,"LogisticRegression(C=0.25, solver='liblinear')",3,0.936208,0.000628,5
6,"LogisticRegression(C=5, l1_ratio=1.0, solver='...",3,0.936205,0.001035,6
3,"LogisticRegression(C=5, solver='liblinear')",3,0.936191,0.001049,7


 - l1 regularization gives better results this time

### Decision Tree - Model Hyperparams

In [46]:
%%time

grid_search_dt_mhp = GridSearchCV(
    pipeline_dt,
    {
        "model__max_depth": range(3, 10),
        "model__max_leaf_nodes": range(10, 100, 10),
    },
    cv=skf,
    scoring="roc_auc",
)

grid_search_dt_mhp.fit(X_raw, y_raw)

CPU times: user 3.25 s, sys: 211 ms, total: 3.46 s
Wall time: 3.49 s


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'model__max_depth': range(3, 10), 'model__max_leaf_nodes': range(10, 100, 10)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",StratifiedKFo...shuffle=False)
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candida

In [47]:
grid_results_ldt_mhp = pd.DataFrame(grid_search_dt_mhp.cv_results_)
grid_results_view_dt_mhp = grid_results_ldt_mhp[
    [
        "param_model__max_depth",
        "param_model__max_leaf_nodes",
        "mean_test_score",
        "std_test_score",
        "rank_test_score",
    ]
].sort_values("mean_test_score", ascending=False)

grid_results_view_dt_mhp

,param_model__max_depth,param_model__max_leaf_nodes,mean_test_score,std_test_score,rank_test_score
30,6,40,0.921909,0.001437,1
39,7,40,0.920436,0.003544,2
40,7,50,0.919526,0.003299,3
49,8,50,0.919312,0.003068,4
58,9,50,0.919212,0.002939,5
...,...,...,...,...,...
5,3,60,0.895825,0.003131,55
4,3,50,0.895825,0.003131,55
3,3,40,0.895825,0.003131,55
2,3,30,0.895825,0.003131,55


## RandomizedSearchCV Experiments

### Decision Tree - Model Hyperparams

In [49]:
%%time

random_search_dt_mhp = RandomizedSearchCV(
    pipeline_dt,
    {
        "model__max_depth": range(3, 20),
        "model__max_leaf_nodes": range(10, 1000, 10),
        "model__min_samples_split": range(2, 20),
        "model__min_samples_leaf": range(1, 20),
        'model__max_features': [None, "sqrt", "log2"],
        'model__criterion': ["gini", "entropy"],
    },
    n_iter=200,
    cv=skf,
    scoring="roc_auc",
)

random_search_dt_mhp.fit(X_raw, y_raw)

CPU times: user 9.37 s, sys: 505 ms, total: 9.88 s
Wall time: 9.93 s


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'model__criterion': ['gini', 'entropy'], 'model__max_depth': range(3, 20), 'model__max_features': [None, 'sqrt', ...], 'model__max_leaf_nodes': range(10, 1000, 10), ...}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",200
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-va

In [50]:
random_results_dt_mhp = pd.DataFrame(random_search_dt_mhp.cv_results_)
random_results_view_dt_mhp = random_results_dt_mhp[
    [
        "param_model__max_depth",
        "param_model__max_leaf_nodes",
        "param_model__min_samples_split",
        "param_model__min_samples_leaf",
        "param_model__max_features",
        "param_model__criterion",
        "mean_test_score",
        "std_test_score",
        "rank_test_score",
    ]
].sort_values("mean_test_score", ascending=False)

random_results_view_dt_mhp.head(20)

,param_model__max_depth,param_model__max_leaf_nodes,param_model__min_samples_split,param_model__min_samples_leaf,param_model__max_features,param_model__criterion,mean_test_score,std_test_score,rank_test_score
143,14,60,2,19,None,gini,0.923499,0.000937,1
27,7,350,16,13,None,gini,0.923048,0.000672,2
2,7,960,12,15,None,gini,0.922780,0.000846,3
56,14,50,7,13,None,gini,0.922675,0.001937,4
80,7,310,8,14,None,gini,0.922454,0.000932,5
41,18,60,2,10,None,entropy,0.920832,0.001548,6
152,5,860,14,18,None,entropy,0.920649,0.000743,7
190,5,60,14,6,None,entropy,0.920563,0.000812,8
69,13,90,9,17,None,gini,0.920548,0.001023,9
64,5,460,12,18,None,gini,0.919724,0.001275,10


### Log Reg - Model Hyperparams and Polinomial degree
#### One more check for 4 polinomial degree

In [59]:
pipeline_lg

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('drop_cols', ...), ('num_flags', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,columns,"['Tenure', 'HasCrCard']"
,flag_config,{}
,mapping,"{'CreditScore': {'bins': [0, 675, ...], 'drop_original': True, 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore'}, 'EstimatedSalary': {'bins': [0, 25000, ...], 'drop_original': True, 'labels': ['low', 'medium', ...], 'new_col': 'SalaryScore'}}"
,flag_config,"{'Geography': [{'drop_original': True, 'flag_name': 'IsGermany', 'value': 'Germany'}]}"
,config,{}
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outpu

In [67]:
%%time

random_search_lg_mhp_pl = RandomizedSearchCV(
    pipeline_lg,
    {
        "preprocessing__num__poly_features__degree": [3,4],
        "model__C": [0.3, 0.5, 1, 5],
        "model__l1_ratio": [0.0, 1.0],
    },
    cv=skf,
    scoring="roc_auc",
    n_iter=10,
)

random_search_lg_mhp_pl.fit(X_raw, y_raw)

CPU times: user 49 s, sys: 458 ms, total: 49.4 s
Wall time: 50.1 s


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...liblinear'))])
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'model__C': [0.3, 0.5, ...], 'model__l1_ratio': [0.0, 1.0], 'preprocessing__num__poly_features__degree': [3, 4]}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",10
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchange

In [68]:
random_results_lg_mhp_pl = pd.DataFrame(random_search_lg_mhp_pl.cv_results_)
random_results_view_lg_mhp_pl = random_results_lg_mhp_pl[
    [
        "param_preprocessing__num__poly_features__degree",
        "param_model__l1_ratio",
        "param_model__C",
        "mean_test_score",
        "std_test_score",
        "rank_test_score",
    ]
].sort_values("mean_test_score", ascending=False)

random_results_view_lg_mhp_pl

,param_preprocessing__num__poly_features__degree,param_model__l1_ratio,param_model__C,mean_test_score,std_test_score,rank_test_score
9,3,1.0,0.3,0.936478,0.000664,1
6,3,1.0,1.0,0.936331,0.000897,2
3,3,0.0,0.5,0.936311,0.000790,3
8,3,0.0,1.0,0.936280,0.000912,4
5,3,0.0,0.3,0.936235,0.000676,5
7,3,0.0,5.0,0.936191,0.001049,6
0,4,1.0,0.3,0.935186,0.000480,7
2,4,0.0,0.3,0.934442,0.000565,8
1,4,0.0,0.5,0.934339,0.000479,9
4,4,0.0,5.0,0.933951,0.000388,10


 - all 3 cases are better than all 4

#### NumericBinner Transformer

In [ ]:
%%time

# fix poly_features__degree at 3
random_search_lg_mhp_nb = RandomizedSearchCV(
    pipeline_lg,
    {
        "num_bin__mapping": [
            {},
            num_bin_mapping,
            {
                "EstimatedSalary": {
                    "bins": [0, 25000, 100000, inf],
                    "labels": ["low", "medium", "high"],
                    "new_col": "SalaryScore",
                    "drop_original": True
                },
            },
            {
                "CreditScore": {
                    "bins": [0, 675, inf],
                    "labels": ["low", "high"],
                    "new_col": "CreditScoreScore",
                    "drop_original": True
                }
            }
        ],
        "preprocessing__num__poly_features__degree": [3],
        "model__C": [0.3, 0.5, 1, 5],
        "model__l1_ratio": [0.0, 1.0],
    },
    cv=skf,
    scoring="roc_auc",
    n_iter=20,
)

random_search_lg_mhp_nb.fit(X_raw, y_raw)

CPU times: user 2min 15s, sys: 1.21 s, total: 2min 16s
Wall time: 2min 18s


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...liblinear'))])
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'model__C': [0.3, 0.5, ...], 'model__l1_ratio': [0.0, 1.0], 'num_bin__mapping': [{}, {'CreditScore': {'bins': [0, 675, ...], 'drop_original': True, 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore'}, 'EstimatedSalary': {'bins': [0, 25000, ...], 'drop_original': True, 'labels': ['low', 'medium', ...], 'new_col': 'SalaryScore'}}, ...], 'preprocessing__num__poly_features__degree': [3]}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",20
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, 

In [71]:
random_results_lg_mhp_nb = pd.DataFrame(random_search_lg_mhp_nb.cv_results_)
random_results_view_lg_mhp_nb = random_results_lg_mhp_nb[
    [
        "param_num_bin__mapping",
        "param_model__l1_ratio",
        "param_model__C",
        "mean_test_score",
        "std_test_score",
        "rank_test_score",
    ]
].sort_values("mean_test_score", ascending=False)

with pd.option_context('display.max_colwidth', None):
    display(
        random_results_view_lg_mhp_nb
    )

,param_num_bin__mapping,param_model__l1_ratio,param_model__C,mean_test_score,std_test_score,rank_test_score
19,"{'EstimatedSalary': {'bins': [0, 25000, 100000, inf], 'labels': ['low', 'medium', 'high'], 'new_col': 'SalaryScore', 'drop_original': True}, 'CreditScore': {'bins': [0, 675, inf], 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore', 'drop_original': True}}",1.0,0.5,0.936452,0.000770,1
1,"{'EstimatedSalary': {'bins': [0, 25000, 100000, inf], 'labels': ['low', 'medium', 'high'], 'new_col': 'SalaryScore', 'drop_original': True}, 'CreditScore': {'bins': [0, 675, inf], 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore', 'drop_original': True}}",0.0,0.5,0.936311,0.000790,2
2,"{'EstimatedSalary': {'bins': [0, 25000, 100000, inf], 'labels': ['low', 'medium', 'high'], 'new_col': 'SalaryScore', 'drop_original': True}, 'CreditScore': {'bins': [0, 675, inf], 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore', 'drop_original': True}}",0.0,1.0,0.936280,0.000912,3
8,"{'EstimatedSalary': {'bins': [0, 25000, 100000, inf], 'labels': ['low', 'medium', 'high'], 'new_col': 'SalaryScore', 'drop_original': True}, 'CreditScore': {'bins': [0, 675, inf], 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore', 'drop_original': True}}",1.0,5.0,0.936196,0.001038,4
5,"{'EstimatedSalary': {'bins': [0, 25000, 100000, inf], 'labels': ['low', 'medium', 'high'], 'new_col': 'SalaryScore', 'drop_original': True}, 'CreditScore': {'bins': [0, 675, inf], 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore', 'drop_original': True}}",0.0,5.0,0.936191,0.001049,5
14,"{'EstimatedSalary': {'bins': [0, 25000, 100000, inf], 'labels': ['low', 'medium', 'high'], 'new_col': 'SalaryScore', 'drop_original': True}}",1.0,0.3,0.936101,0.000810,6
17,"{'CreditScore': {'bins': [0, 675, inf], 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore', 'drop_original': True}}",1.0,0.3,0.935892,0.000553,7
7,"{'EstimatedSalary': {'bins': [0, 25000, 100000, inf], 'labels': ['low', 'medium', 'high'], 'new_col': 'SalaryScore', 'drop_original': True}}",1.0,1.0,0.935846,0.001074,8
3,"{'EstimatedSalary': {'bins': [0, 25000, 100000, inf], 'labels': ['low', 'medium', 'high'], 'new_col': 'SalaryScore', 'drop_original': True}}",0.0,0.5,0.935766,0.000996,9
15,"{'CreditScore': {'bins': [0, 675, inf], 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore', 'drop_original': True}}",1.0,0.5,0.935756,0.000740,10


 - EstimatedSalary and CreditScore are better if customly transformed than included full data

#### Model Hyperparams

In [73]:
%%time

# fix poly_features__degree at 3
random_search_lg_mhp = RandomizedSearchCV(
    pipeline_lg,
    {
        "preprocessing__num__poly_features__degree": [3],
        "model__C": [0.2, 0.3, 0.5, 1, 5],
        "model__l1_ratio": [0.0, 1.0],
    },
    cv=skf,
    scoring="roc_auc",
    n_iter=10,
)

random_search_lg_mhp.fit(X_raw, y_raw)

CPU times: user 39.9 s, sys: 376 ms, total: 40.3 s
Wall time: 40.8 s


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...liblinear'))])
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'model__C': [0.2, 0.3, ...], 'model__l1_ratio': [0.0, 1.0], 'preprocessing__num__poly_features__degree': [3]}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",10
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged::

In [74]:
random_results_lg_mhp = pd.DataFrame(random_search_lg_mhp.cv_results_)
random_results_view_lg_mhp = random_results_lg_mhp[
    [
        "param_model__l1_ratio",
        "param_model__C",
        "mean_test_score",
        "std_test_score",
        "rank_test_score",
    ]
].sort_values("mean_test_score", ascending=False)

random_results_view_lg_mhp.head(10)

,param_model__l1_ratio,param_model__C,mean_test_score,std_test_score,rank_test_score
3,1.0,0.3,0.936476,0.000665,1
5,1.0,0.5,0.936454,0.000772,2
1,1.0,0.2,0.936431,0.000680,3
7,1.0,1.0,0.936326,0.000897,4
4,0.0,0.5,0.936311,0.000790,5
6,0.0,1.0,0.936280,0.000912,6
2,0.0,0.3,0.936235,0.000676,7
9,1.0,5.0,0.936201,0.001031,8
8,0.0,5.0,0.936191,0.001049,9
0,0.0,0.2,0.936155,0.000570,10


 - l1 is better than l2 this time
 - l1 with with C=0.3  is the best

## Test Data Prediction

In [76]:
test_raw_df = pd.read_csv("downloads/test.csv")
test_raw_df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15000,15594796.0,Chu,584.0,Germany,Male,30.0,2.0,146053.66,1.0,1.0,1.0,157891.86
1,15001,15642821.0,Mazzi,551.0,France,Male,39.0,5.0,0.00,2.0,1.0,1.0,67431.28
2,15002,15716284.0,Onyekachi,706.0,France,Male,43.0,8.0,0.00,2.0,1.0,0.0,156768.45
3,15003,15785078.0,Martin,717.0,Spain,Male,45.0,3.0,0.00,1.0,1.0,1.0,166909.87
4,15004,15662955.0,Kenechukwu,592.0,Spain,Male,43.0,8.0,0.00,2.0,1.0,1.0,143681.97


### Decision Tree

In [75]:
best_dt_model_pipeline = random_search_dt_mhp.best_estimator_
best_dt_model_pipeline

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('drop_cols', ...), ('preprocessing', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,columns,[]
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the dif

In [77]:
test_pred_proba = best_dt_model_pipeline.predict_proba(test_raw_df)[:, 1]
test_raw_df["Exited"] = test_pred_proba.round(2)
test_raw_df["Exited"].head()

0    0.22
1    0.01
2    0.01
3    0.51
4    0.01
Name: Exited, dtype: float64

In [78]:
sample_submission_df = pd.read_csv("downloads/sample_submission.csv")
sample_submission_df["Exited"] = test_raw_df["Exited"]
sample_submission_df.to_csv("downloads/submission_decision_tree_best.csv", index=False)

### Logistic Regression

In [80]:
best_lg_model_pipeline = random_search_lg_mhp.best_estimator_
best_lg_model_pipeline

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('drop_cols', ...), ('num_flags', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,columns,"['Tenure', 'HasCrCard']"
,flag_config,{}
,mapping,"{'CreditScore': {'bins': [0, 675, ...], 'drop_original': True, 'labels': ['low', 'high'], 'new_col': 'CreditScoreScore'}, 'EstimatedSalary': {'bins': [0, 25000, ...], 'drop_original': True, 'labels': ['low', 'medium', ...], 'new_col': 'SalaryScore'}}"
,flag_config,"{'Geography': [{'drop_original': True, 'flag_name': 'IsGermany', 'value': 'Germany'}]}"
,config,{}
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outpu

In [81]:
test_pred_proba = best_lg_model_pipeline.predict_proba(test_raw_df)[:, 1]
test_raw_df["Exited"] = test_pred_proba.round(2)
test_raw_df["Exited"].head()

0    0.09
1    0.01
2    0.05
3    0.44
4    0.03
Name: Exited, dtype: float64

In [82]:
sample_submission_df = pd.read_csv("downloads/sample_submission.csv")
sample_submission_df["Exited"] = test_raw_df["Exited"]
sample_submission_df.to_csv("downloads/submission_poly_reg_optimal_v4.csv", index=False)